# Spot detection

**Goal:** Iterate through a COCO-format dataset, crop each annotated bounding box from its image, and save the crops.

**Inputs**
- `annotations.json` (COCO format) with `images` and `annotations` sections
- Image files located under `image_dir`

**Outputs**
- Cropped image files saved under `output_dir`, named with image base name + annotation id + category id


## Title and Scope <a id="env"></a>




# speck_report_mm^2 — Region of Interest & Dark-Speck Area Report (mm²)

**What this does**
- For each handsheet image and its paired LabelMe JSON, compute:
  - Sheet area scale (mm² per pixel) from a known sheet diameter
  - ROI rectangle areas (px and mm²) for labels starting with `S`, `p`, `r`, or `1`
  - Dark-speck area within each ROI using Otsu thresholding (px and mm²)
- Save each ROI crop to `./cropped_rois_<filter>/<base>.png`
- Save each masked speck visualization to `./masked_specks_<filter>/<base>_mask.png`
- Append results to `speck_report_mm2_with_cropped_images_<filter>.csv`

**Assumptions**
- Images (`.jpg/.jpeg/.png`) and LabelMe JSON files live in the same folder.
- Each JSON name starts with the image’s stem (e.g., `sheet_01.json` for `sheet_01.png`).
- LabelMe rectangles have `shape_type="rectangle"` and labels like `S1`, `p02`, `r`, or `1`.

In [ ]:
from pathlib import Path
import json, csv, cv2, numpy as np, math

#constants
D_MM = 164.4                                    # true sheet diameter (mm)
A_SHEET_MM2 = math.pi * (D_MM / 2) ** 2         # real sheet area (mm²)

folder       = Path("Area")                     # images + JSON here
#crop_dir     = folder / "cropped_rois_astNlMeansDenoising"
#mask_dir     = folder / "masked_specks_astNlMeansDenoising"
#crop_dir     = folder / "cropped_rois_pyrMeanShiftFiltering"
#mask_dir     = folder / "masked_specks_pyrMeanShiftFiltering"
#crop_dir     = folder / "cropped_rois_medianBlur"
#mask_dir     = folder / "masked_specks_medianBlur"
#crop_dir     = folder / "cropped_rois_GaussianBlur"
#mask_dir     = folder / "masked_specks_GaussianBlur"
crop_dir     = folder / "cropped_rois_bilateralFilter"
mask_dir     = folder / "masked_specks_bilateralFilter"
#out_csv      = "speck_report_mm2_with_cropped_images_pyrMeanShiftFiltering.csv"
#out_csv      = "speck_report_mm2_with_cropped_images_astNlMeansDenoising.csv"
#out_csv      = "speck_report_mm2_with_cropped_images_medianBlur.csv"
#out_csv      = "speck_report_mm2_with_cropped_images_GaussianBlur.csv"
out_csv      = "speck_report_mm2_with_cropped_images_bilateralFilter.csv"

crop_dir.mkdir(exist_ok=True)
mask_dir.mkdir(exist_ok=True)
COLOR      = (0, 0, 255)    # BGR → red; change to (0,255,255)=yellow and etc.
ALPHA      = 0.5  
rows = []                   # CSV rows collected here


## Dark-Speck Segmentation (Otsu Threshold on Gray)

Segmentation of dark specks inside each region of interest using grayscale + Otsu with an **inverse** binary threshold:
- Converts ROI to grayscale
- Chooses threshold automatically (Otsu)
- Inverts so darker specks → 255 (foreground), background → 0

Returns a **binary** `uint8` mask:
- 255 = speck pixels
- 0   = background


In [ ]:
#create binary mask of dark speck in Region of interest
def mask_dark_speck(roi_bgr):
    gray = cv2.cvtColor(roi_bgr, cv2.COLOR_BGR2GRAY)
    _, mask = cv2.threshold(
        gray, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU
    )
    return mask  # uint8: 255 = speck, 0 = background


## Main Pass Over Images & LabelMe JSON

For each image:
1. Find a JSON whose name starts with the image stem (skip if none).
2. Read image size → compute `mm² per pixel` using known sheet area.
3. Iterate LabelMe shapes, keeping **rectangles** whose label starts with `S`, `p`, `r`, or `1`.
4. Clip Region of interest to image bounds; compute:
   - `roi_px` and `roi_mm2`
   - Apply chosen smoothing filter (uncomment the variants needed)
   - `mask = mask_dark_speck(roi)` → `dark_px` & `dark_mm2`
5. Save:
   - Region of interest crop → `cropped_rois_<filter>/<base>.png`
   - Visualization with masked speck overlay → `masked_specks_<filter>/<base>_mask.png`
6. Append a row to the CSV buffer.

Finally, write `out_csv` with per-ROI metrics.


In [ ]:
if __name__=='__main__':
    #iterate over every image file 
    for img_path in folder.iterdir():
        if img_path.suffix.lower() not in (".jpg", ".jpeg", ".png"):
            continue

        # find any JSON whose name begins with the image’s stem
        json_matches = list(folder.glob(f"{img_path.stem}*.json"))
        if not json_matches:
            print(f"[skip] {img_path.name} → no JSON")
            continue
        json_path = json_matches[0]

        img = cv2.imread(str(img_path))
        if img is None:
            print(f"[warn] cannot read {img_path.name}")
            continue

        H, W   = img.shape[:2]
        sheet_px = H * W
        D_px = (W + H) / 2              # diameter in pixels
        sheet_px = math.pi * (D_px / 2) ** 2     # area of that circle (px²)
        mm2_per_px = A_SHEET_MM2 / sheet_px          # scale factor for this image

        with open(json_path) as f:
            ann = json.load(f)

        # index rectangles so filenames stay unique even if labels repeat
        rect_counter = 0

        for shp in ann["shapes"]:
            if (
                shp.get("shape_type") != "rectangle"
                or not shp["label"]
                or shp["label"][0] not in ("S", "p", "r", '1')
            ):
                continue

            rect_counter += 1
            label = shp["label"]

            # rectangle corners
            (x1, y1) = map(int, shp["points"][0])
            (x2, y2) = map(int, shp["points"][1])
            x1, x2 = sorted((max(0, x1), min(W, x2)))
            y1, y2 = sorted((max(0, y1), min(H, y2)))

            roi      = img[y1:y2, x1:x2]
            roi_px   = roi.shape[0] * roi.shape[1]
            roi_mm2  = roi_px * mm2_per_px

            # choose ONE smoothing prefilter (uncomment the one you want)
            #roi = cv2.pyrMeanShiftFiltering(roi, 6, 10)
            #roi = cv2.fastNlMeansDenoising(roi, None, 15, 7, 25) 
            #roi = cv2.medianBlur(roi, 3)
            #roi = cv2.GaussianBlur(roi, (3, 3), 0)
            roi = cv2.bilateralFilter(roi,        # input image
                                      d=10,       # pixel-neighbourhood diameter
                                      sigmaColor=40,   # larger means stronger colour smoothing
                                      sigmaSpace=20)

            mask = mask_dark_speck(roi)

            # (optional) morphology 
            k = np.ones((5, 5), np.uint8)
            #mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN,  k, 2)
            #mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, k, 2)
            #num, lbl, stats, _ = cv2.connectedComponentsWithStats(mask)
            #if num > 1:
            #    largest = 1 + np.argmax(stats[1:, cv2.CC_STAT_AREA])
            #    mask = np.where(lbl == largest, 255, 0).astype("uint8")

            dark_px  = int(np.count_nonzero(mask))
            dark_mm2 = dark_px * mm2_per_px

            #save ROI and masked speck images 
            base_name = f"{img_path.stem}_{label}_{rect_counter}"
            cv2.imwrite(str(crop_dir / f"{base_name}.png"), roi)

            # masked image: keep only speck pixels, black elsewhere (with overlay)
            masked_roi = cv2.bitwise_and(roi, roi, mask=mask)
            overlay = roi.copy()
            overlay[mask == 255] = COLOR
            viz = cv2.addWeighted(overlay, ALPHA, roi, 1-ALPHA, 0)
            cv2.imwrite(str(mask_dir / f"{base_name}_mask.png"), viz)

            #collect CSV row
            rows.append([
                img_path.name,
                label,
                img_px,
                round(A_SHEET_MM2, 2),
                roi_px,
                round(roi_mm2, 2),
                dark_px,
                round(dark_mm2, 2),
            ])

    # write the CSV
    with open(out_csv, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow([
            "image", "label",
            "img_px",  "img_mm2",
            "roi_px",  "roi_mm2",
            "dark_px", "dark_mm2"
        ])
        writer.writerows(rows)

    print(f"✓ Done.\n  • Crops saved to      {crop_dir}\n  • Masks saved to     {mask_dir}\n  • Report saved as    {out_csv}")
